## 1. Import Required Packages

In [1]:
# Import required packages
from ethernet.client import Client

## 2. Connect to EtherNet Core

In [2]:
# Connect to EtherNet Core
ec = Client(
    core_host="192.168.1.99",
    core_grpc_port=9090,
    core_http_port=8080
)

## 3. List the Neo4j Databases

In [3]:
# List the Neo4j databases available
ec.get_databases()

[]

## 4. Create Neo4j Graphs


## 4.1.  Create a `token_transfers` graph

Create a `token_transfers` graph for block range [2000000, 2000500]

- Start block_number `2_000_000`
- End block_number `2_000_500`

Both start and end range are **inclusive**

In [4]:
# Create a token_transfers graph for block range `[2000000, 2000500]`

response_0 = ec.create_token_transfers_graph(2000000, 2000500)
response_0

databaseName: "token-transfers-2000000-2000500-20210612t1522z.db"

## 5. List the Neo4j Databases Again

This is to confirm that the database has indeed been created.

In [5]:
# List the Neo4j databases available
dbs = ec.get_databases()
dbs

['token-transfers-2000000-2000500-20210612t1522z.db']

## 6. Switch Neo4j Database

Switch to the Neo4j database/graph that has just been created.

In [8]:
# Switch database
ec.switch_database(dbs[0])

## 7. Navigate to the Neo4j Web UI

If you would like to work on the graphs that you have created through the Neo4j Web UI, please use the link below.

**Note:** You will have to modify the `neo4j_server` address to point to the `ip_address` that your Neo4j instance is being hosted on.

<a>http://neo4j_server:7474/browser/</a>

The credentials that we have deployed the Neo4j instance are:
```
username: neo4j
password: test
```

## 8. Constructing other graphs (Additional)

EtherNet also allows user to construct `traces` and `transactions` graph.

The code blocks below demonstrates how a user can construct these graphs.

## 8.1. Constructing `traces` graph
The code below allows users to construct `traces` graphs.

In [ ]:
response_1 = ec.create_traces_graph(2000000, 2000500)
response_1

## 8.2. Constructing `transactions` graph
The code below allows users to construct `transactions` graphs.

In [ ]:
response_2 = ec.create_transactions_graph(2000000, 2000500)
response_2

## 9. Analysis

In this section, we will demonstrate how a user can analyse the `token_transfers` graph created from `step 4.1`.

In this simple analysis demonsitration, we will be using the [Neo4j Graph Data Science Library](https://neo4j.com/docs/graph-data-science/current/) to extract some graph characteristics/metrics.

We will mainly be demonstrating how one can:

1. Get the degree-centrality (in-degree) of a graph
2. Get the degree-centrality (out-degree) of a graph
3. Get the strongly connected component metrics of a graph 
4. Determine which strongly connected component each address/node belongs to of a graph 


### 9.1. Install dependencies

First install the required dependencies.

In [9]:
# Install Neo4j driver
!pip3 install neo4j
# Install pandas
!pip3 install pandas

### 9.2. Declare helper classes to connect to Neo4j

Declare a helper class to connect and submit queries to Neo4j easily.

In [10]:
import pandas as pd
from neo4j import __version__ as neo4j_version

# Set maximum number of rows to be displayed
pd.set_option("display.max_rows", 100)

# Print Neo4j version
print(f"Python Neo4j module version: {neo4j_version}")

from neo4j import GraphDatabase

# Declare a helper class to connect to Neo4j

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

Python Neo4j module version: 4.3.1


### 9.3. Connect to Neo4j

To connect to Neo4j, one can fetch the Neo4j connection details from EtherNet core

In [11]:
neo4j_config = ec.get_neo4j_config()
neo4j_config

connection_uri: "bolt://192.168.1.99:7687"
username: "neo4j"
password: "test"

In [12]:
# Connect to Neo4j
conn = Neo4jConnection(
    uri=neo4j_config.connection_uri, 
    user=neo4j_config.username, 
    pwd=neo4j_config.password
)

In [13]:
# Declare graph we are going to work with 
print(dbs)
db = dbs[0]

['token-transfers-2000000-2000500-20210612t1522z.db']


### 9.4. Screenshot of demo graph
Before we demonstrate some simple queries to extract graph properties, let us visualise the graph we are working with.
![graph_svg](demo_resources/token_transfer_2000000_2000500.svg)

### 9.5. Degree centrality of nodes (Out-degree)
Degree centrality measures the number of incoming or outgoing (or both) relationships from a node, depending on the orientation of a relationship projection. 

It can be applied to either weighted or unweighted graphs. In the weighted case the algorithm computes the sum of all positive weights of adjacent relationships of a node, for each node in the graph.

For the sake of simplicty, we are going to assume that the graph is **unweighted**, keeping this example simple for demonstration purposes.

In the example below, we can find the number of `outgoing_transfers` each address has for the blocks between `2000000` and `2000500`.

In [14]:
query_string = """
CALL gds.degree.stream({
    nodeProjection: 'Address',
    relationshipProjection: 'TOKEN_TRANSFER'
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).address_string AS address, score AS outgoing_transfers
ORDER BY outgoing_transfers DESC, address DESC
"""

dtf_data_0 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_0

,address,outgoing_transfers
0,0xdf21fa922215b1a56f5a6d6294e6e36c85a0acfb,8.0
1,0xafa907a69895898d42370354a7d7e5e1fab654c2,7.0
2,0x78be431bd9c55c60689709b90547db65f3cda563,6.0
3,0x147af46ae9ccd18bb35ca01b353b51990e49dce1,4.0
4,0xec9f6c9634165f91e22e58b90e3ede393d959e47,3.0
5,0xc350d1ecc1c71e41612be8c6c690f8af4dda32e6,3.0
6,0xe9d3db337f5e3d722a41b70745eef56ee99ecace,2.0
7,0x6b95702ea15d990a30b31376c18a7d31d3d755fe,2.0
8,0x0a869d79a7052c7f1b55a8ebabbea3420f0d1e13,2.0
9,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,1.0


### 9.5.1. Interpreting the results obtained
From the results above, we can see that the address `0xdf21fa922215b1a56f5a6d6294e6e36c85a0acfb` **made the most** (8 transfers made) number of token transfers to other addresses between the blocks `2000000` and `2000500`.

### 9.6. Degree centrality of nodes (In-degree)

In the example below, we can find the number of `incoming_transfers` each address has for the blocks between `2000000` and `2000500`.

In [15]:
query_string = """
CALL gds.degree.stream({
    nodeProjection: 'Address',
    relationshipProjection: 'TOKEN_TRANSFER',
    orientation: 'REVERSE'
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).address_string AS address, score AS incoming_transfers
ORDER BY incoming_transfers DESC, address DESC
"""

dtf_data_1 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_1

,address,incoming_transfers
0,0xbf4ed7b27f1d666546e30d74d50d173d20bca754,22.0
1,0xec9f6c9634165f91e22e58b90e3ede393d959e47,8.0
2,0xafa907a69895898d42370354a7d7e5e1fab654c2,6.0
3,0x32be343b94f860124dc4fee278fdcbd38c102d88,5.0
4,0xdf21fa922215b1a56f5a6d6294e6e36c85a0acfb,2.0
5,0xab8d8b74f202f4cd4a918b65da4bac612e086ee7,2.0
6,0x86d7d21b0f006da9d27fac5dc359be8bfa990c2d,2.0
7,0x78be431bd9c55c60689709b90547db65f3cda563,2.0
8,0x6bb740b57627123ecb2debe195ea5b225b4f9ad8,2.0
9,0x0a869d79a7052c7f1b55a8ebabbea3420f0d1e13,2.0


### 9.6.1. Interpreting the results obtained
From the results above, we can see that the address `0xbf4ed7b27f1d666546e30d74d50d173d20bca754` **received the most** (22 transfers received) number of token transfers to other addresses between the blocks `2000000` and `2000500`.

## 9.7. Get the strongly connected component (SCC) metrics

The Strongly Connected Components (SCC) algorithm finds maximal sets of connected nodes in a directed graph. A set is considered a strongly connected component if there is a directed path between each pair of nodes within the set. It is often used early in a graph analysis process to help us get an idea of how our graph is structured.

In [16]:
query_string = """
CALL gds.alpha.scc.write({
    nodeProjection: 'Address',
    relationshipProjection: 'TOKEN_TRANSFER',
    writeProperty: 'componentId'
})
YIELD setCount AS set_count, maxSetSize AS max_set_size, minSetSize AS min_set_size;
"""

dtf_data_2 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_2

,set_count,max_set_size,min_set_size
0,67,4,1


### 9.7.1. How to interpret the results obtained

Out of the **72** addresses/nodes that were involved in token transfers between blocks `2000000` and `2000500`, there are **67** SCC sets.

Out of the **67** SCC sets, the **largest** component set has **4** addresses forming this SCC set. 

Out of the **67** SCC sets, the **smallest** component set has **1** address forming this SCC set.

## 9.8. Get the composition of the SCC sets

Out of the **67** SCC sets, we are also able to determine the specific nodes/addresses that fall into the different SCC sets.

In [17]:
query_string = """
CALL gds.alpha.scc.stream({
    nodeProjection: 'Address',
    relationshipProjection: 'TOKEN_TRANSFER'
})
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).address_string AS address, componentId AS component_idx
ORDER BY component_idx DESC
"""

dtf_data_3 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_3

,address,component_idx
0,0x9a63d185a79129fdab19b58bb631ea36a420544e,72
1,0xf9dff387dcb5cc4cca5b91adb07a95f54e9f1bb6,71
2,0xab8d8b74f202f4cd4a918b65da4bac612e086ee7,70
3,0x6e39564ecfd4b5b0ba36cd944a46bca6063cace5,68
4,0xe8df9a7c34736a482a861a49b51fbc1c4c031456,67
5,0x00995ae79d60365b2a0c52a5ad19140dfb8d45f2,66
6,0x45d01cc478a3dcfd66607726dc50780f3eb4d7bc,65
7,0xcaac46d9bd68bffb533320545a90cd92c6e98e58,64
8,0x92336742d742b6e684152979f4fa829aaa495d66,63
9,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,62


## 9.8.1. How interpret the results obtained

As observed in the results above, the nodes/addresses:

```
1. 0xdf21fa922215b1a56f5a6d6294e6e36c85a0acfb
2. 0x21dc969a31e4a66aec4775c54ae47a783aa9f9d9
3. 0x0a869d79a7052c7f1b55a8ebabbea3420f0d1e13
4. 0x86d7d21b0f006da9d27fac5dc359be8bfa990c2d
```

makes up the largest SCC set of index `10`.


Below is a screenshot of the subgraph that makes up this strongly connected component.
![scc_subgraph](demo_resources/scc_subgraph.png)

### 9.9. A more specific example
A `token_transfers` graph may contain token transfers between addresses of more than 1 type of token.

This is the case for the graph that we have created in [4.1. Create a token_transfers graph](#4.1.--Create-a-token_transfers-graph).

We can show this by querying out the different token addresses that are involved in the `token_transfers` graph between blocks `2000000` and `2000500`.

**Note:** Some addresses might partake in token transfers in more than 1 token network in the graph.

In [18]:
query_string = """
MATCH (a:Address)-[r:TOKEN_TRANSFER]-(b:Address)
RETURN distinct(r.token_address) AS token_address, COUNT(DISTINCT(a.address_string)) AS addresses_involved
"""

dtf_data_4 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_4.head(dtf_data_3.shape[0])

# Function to make link clickable
def make_clickable(val: str):
    return '<a href="https://etherscan.io/address/{}">{}</a>'.format(val, val)

# Add a link to the token
dtf_data_4["token_details"] = dtf_data_4.token_address
dtf_data_4 = dtf_data_4.style.format({"token_details": make_clickable})
dtf_data_4

,token_address,addresses_involved,token_details
0,0x57d90b64a1a57749b0f932f1a3395792e12e7055,2,0x57d90b64a1a57749b0f932f1a3395792e12e7055
1,0xa0dcc980a9ed1fbeb6641b098929ee7caad5c5a6,3,0xa0dcc980a9ed1fbeb6641b098929ee7caad5c5a6
2,0xbb9bc244d798123fde783fcc1c72d3bb8c189413,39,0xbb9bc244d798123fde783fcc1c72d3bb8c189413
3,0xecf8f87f810ecf450940c9f60066b4a7a501d6a7,7,0xecf8f87f810ecf450940c9f60066b4a7a501d6a7
4,0xc66ea802717bfb9833400264dd12c2bceaa34a6d,6,0xc66ea802717bfb9833400264dd12c2bceaa34a6d
5,0xe0b7927c4af23765cb51314a0e0521a9645f0e2a,18,0xe0b7927c4af23765cb51314a0e0521a9645f0e2a


From the results obtained above, there are **5** token networks in this graph, with the largest network belonging to `0xbb9bc244d798123fde783fcc1c72d3bb8c189413` - **TheDAO (TheDAO)** token.

We can filter our graph to only focus only on **TheDAO (TheDAO)** network and apply the `SCC metrics` and `SCC composition` Cypher query using projection as shown below.

In [19]:
subquery_string = """
MATCH (a:Address)-[r:TOKEN_TRANSFER]-(:Address) 
WHERE r.token_address = "0xbb9bc244d798123fde783fcc1c72d3bb8c189413"
""".replace('\r', '').replace('\n', '')  # Subquery cannot have line breaks

query_string = f"""
CALL gds.alpha.scc.write({{
    nodeQuery: 'MATCH (a:Address) WHERE EXISTS {{{subquery_string}}} RETURN id(a) AS id',
    relationshipQuery: 'MATCH (a1:Address)-[r:TOKEN_TRANSFER]->(a2:Address) WHERE EXISTS {{{subquery_string}}} RETURN id(a1) AS source, id(a2) AS target',
    writeProperty: 'componentId'
}})
YIELD setCount AS set_count, maxSetSize AS max_set_size, minSetSize AS min_set_size;
"""

dtf_data_5 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_5

,set_count,max_set_size,min_set_size
0,36,4,1


In [20]:
query_string = f"""
CALL gds.alpha.scc.stream({{
    nodeQuery: 'MATCH (a:Address) WHERE EXISTS {{{subquery_string}}} RETURN id(a) AS id',
    relationshipQuery: 'MATCH (a1:Address)-[r:TOKEN_TRANSFER]->(a2:Address) WHERE EXISTS {{{subquery_string}}} RETURN id(a1) AS source, id(a2) AS target'
}})
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).address_string AS address, componentId AS component_idx
ORDER BY component_idx DESC
"""

dtf_data_6 = pd.DataFrame([dict(_) for _ in conn.query(query_string, db=db)])
dtf_data_6

,address,component_idx
0,0x21d969141d9577539f83b4e643630761b1eba872,38
1,0xe68d2582d54cd865c8fecd9733525d60be68af67,37
2,0x115e8bec26adb5751540f0f376c2ea809084e211,36
3,0x385d810913b7c436c89a5a639f9b4dd94944e30a,35
4,0x1ccd28d03d7f97a2c3d1d5231242d4cdea28552b,34
5,0xe43e41b65400c8f5da681feb34710bd7552b95de,33
6,0x53a90779bd1ac85d65b407fa9aa03e1106c140d6,32
7,0x128f3e0181082e6f23a1259348c559040526c300,31
8,0xc135091910533b08cb99c4dbdfa19439c13ab92a,30
9,0x5eceae144f4ef84833ce97a41b0647db55350443,29


As we are working on a "sub-graph" of the `token_transfers` graph created from blocks `2000000` and `2005000`, we should see similar results.

The purpose of this example is to show how one can easily break down a large graph into smaller sub-graph.
This can be done by filtering a large graph using attributes like:
1. token_address (if one is working on a token_transfers graph)
2. block_number
3. block_timestamp


## 9.10. Close the Neo4j connection

If you have completed your analysis, you can close the Neo4j connection that was opened in [9.3. Connect to Neo4j](#9.3.-Connect-to-Neo4j).

In [21]:
# Close the Neo4j connection
conn.close()